In [2]:
import pandas as pd
import os
import numpy as np

carpeta_script = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Experimentos en clase\\Scripts'
carpeta_resultados = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Experimentos en clase\\Resultados'
carpeta_datos = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Experimentos en clase\\Datos'


In [3]:
#Carga archivos
sell_in = pd.read_csv(os.path.join(carpeta_datos, 'sell-in.txt'), delimiter = '\t')
tb_productos = pd.read_csv(os.path.join(carpeta_datos, 'tb_productos.txt'), delimiter = '\t')
tb_stocks = pd.read_csv(os.path.join(carpeta_datos, 'tb_stocks.txt'), delimiter = '\t')
productos_predecir = pd.read_csv(os.path.join(carpeta_datos, 'productos_a_predecir.txt'), delimiter = '\t')  

In [4]:
sell_in.head()
productos_predecir.head()

ventas_producto_mes = sell_in.groupby(['periodo', 'product_id'])['tn'].sum()
ventas_producto_mes = ventas_producto_mes.reset_index()
ventas_producto_mes['Timestamp'] = pd.to_datetime(ventas_producto_mes['periodo'], format='%Y%m')
ventas_producto_mes.set_index('Timestamp', inplace=True)
ventas_producto_mes.drop(columns=['periodo'], inplace = True)

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler


def crear_dataset_supervisado(array, input_length, output_length):

    # Inicialización
    X, Y = [], []    # Listados que contendrán los datos de entrada y salida del modelo
    shape = array.shape
    if len(shape)==1: # Si tenemos sólo una serie (univariado)
        fils, cols = array.shape[0], 1
        array = array.reshape(fils,cols)
    else: # Multivariado
        fils, cols = array.shape

    # Generar los arreglos (utilizando ventanas deslizantes de longitud input_length)
    for i in range(fils-input_length-output_length + 1):
        X.append(array[i:i+input_length,0:cols])
        Y.append(array[i+input_length:i+input_length+output_length,-1].reshape(output_length,1))

    # Convertir listas a arreglos de NumPy
    X = np.array(X)
    Y = np.array(Y)

    return X, Y

def diferenciar(dataset, intervalo=1):
    diff = list()
    for i in range(intervalo, len(dataset)):
        value = dataset[i] - dataset[i - intervalo]
        diff.append(value[0])
    return pd.Series(diff)

def invertir_diferenciar(history, yhat, intervalo=1):
    return yhat + history[-intervalo]

def predecir(x, model, scaler, history):
    # Calcular predicción escalada en el rango de escalado
    y_pred_s = model.predict(x,verbose=0)
    # Llevar la predicción a la escala original
    y_pred = scaler.inverse_transform(y_pred_s)
    y_pred = invertir_diferenciar(history, y_pred, 1)
    return y_pred.flatten()

ventana_input = 12
ventana_output = 2

lista_productos_LSTM = []
lista_predicciones_LSTM = []
scaler_list = []

i = 0
for producto in ventas_producto_mes['product_id'].unique():
    if producto in list(productos_predecir['product_id']):
        print(f'Entrenando producto {i}')
        i = i + 1
        ventas_mes_por_producto = ventas_producto_mes[ventas_producto_mes['product_id'] == producto].copy()
        ventas_mes_por_producto.drop(columns=['product_id'], inplace = True)
        ventas_mes_por_producto_values = ventas_mes_por_producto.values
        ventas_mes_por_producto_diferenciado = diferenciar(ventas_mes_por_producto_values, 1)
        #Escalar valor
        scaler = StandardScaler()
        ventas_mes_por_producto_escalado = scaler.fit_transform(ventas_mes_por_producto_diferenciado.values.reshape(-1,1))
        scaler_list.append(scaler)
        try:
            #Formatear valores para input LSTM
            X, Y =crear_dataset_supervisado(ventas_mes_por_producto_escalado, ventana_input, ventana_output)

            # Create and fit the LSTM network
            model = Sequential()
            model.add(LSTM(50, return_sequences=True, input_shape=(ventana_input, 1)))
            model.add(LSTM(50))
            model.add(Dense(ventana_output))
            model.compile(loss='mean_squared_error', optimizer='adam')
            model.fit(X, Y, epochs=100, batch_size=1, verbose=0)

            #Predecir valores
            prediccion_mes_2 = predecir(X[-1].reshape(1,-1), model, scaler, ventas_mes_por_producto_values)[1]

            lista_productos_LSTM.append(producto)
            lista_predicciones_LSTM.append(prediccion_mes_2)
        except:
            print('Valores insuficientes para usar ventana de 12 para LSTM, se predice por promedio')
            lista_productos_LSTM.append(producto)
            lista_predicciones_LSTM.append(ventas_mes_por_producto['tn'].mean())

Entrenando producto 0
Entrenando producto 1
Entrenando producto 2
Entrenando producto 3
Entrenando producto 4
Entrenando producto 5
Entrenando producto 6
Entrenando producto 7
Entrenando producto 8
Entrenando producto 9
Entrenando producto 10
Entrenando producto 11
Entrenando producto 12
Entrenando producto 13
Entrenando producto 14
Entrenando producto 15
Entrenando producto 16
Entrenando producto 17
Entrenando producto 18
Entrenando producto 19
Entrenando producto 20
Entrenando producto 21
Entrenando producto 22
Entrenando producto 23
Entrenando producto 24
Entrenando producto 25
Entrenando producto 26
Entrenando producto 27
Entrenando producto 28
Entrenando producto 29
Entrenando producto 30
Entrenando producto 31
Entrenando producto 32
Entrenando producto 33
Entrenando producto 34
Entrenando producto 35
Entrenando producto 36
Entrenando producto 37
Entrenando producto 38
Entrenando producto 39
Entrenando producto 40
Entrenando producto 41
Entrenando producto 42
Entrenando producto 4

In [7]:
resultados_kaggle_LSTM = pd.DataFrame({'product_id': lista_productos_LSTM, 'tn': lista_predicciones_LSTM})
resultados_kaggle_LSTM['tn'] = resultados_kaggle_LSTM['tn'].apply(lambda x: max(0,x))
resultados_kaggle_LSTM.to_csv(os.path.join(carpeta_resultados, 'predicciones_LSTM_diferenciacion.csv'), index= False)